In [1]:
import tensorflow as tf
import pandas as pd
from setting import *
from model import *

In [2]:
metadata = pd.read_csv(data+'metadata.csv',sep='|',header=None)
print(metadata.shape)
metadata.head()

(13100, 3)


,0,1,2
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [3]:
paths, lengths, texts = [], [], []
for i in range(metadata.shape[0]):
    paths.append('%swavs/%s.wav'%(data, metadata.iloc[i,0]))
    text, converted = text2idx(metadata.iloc[i,1])
    texts.append(np.array(converted, np.int32))
    lengths.append(len(text))

In [4]:
count_correct = 0
mel_list = [os.path.basename(i)[:-4] for i in os.listdir('spectrogram')]
for i in metadata.iloc[:, 0].tolist():
    if i in mel_list:
        count_correct += 1

if count_correct == metadata.shape[0]:
    print('load from cached')
    get_function = get_cached
else:
    print('load from files')
    get_function = load_file

load from cached


In [ ]:
def dynamic_batching(paths):
    files, max_x = [], 0
    for n in range(batch_size):
        files.append(get_function(paths[n]))
        if files[-1][1].shape[0] > max_x:
            max_x = files[-1][1].shape[0]
    return files, max_x

In [ ]:
from tqdm import tqdm
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
batching = (len(paths) // batch_size) * batch_size
EPOCH = 100
for i in range(EPOCH):
    for k in tqdm(range(0, batching, batch_size),desc='minibatch loop'):
        files, max_x = dynamic_batching(paths[k:k+batch_size])
        max_y = max(lengths[k:k+batch_size])
        batch_x = np.zeros((batch_size, max_x, n_mels*reduction_factor))
        batch_y = np.zeros((batch_size, max_y))
        for n in range(batch_size):
            batch_x[n, :, :] = np.pad(files[n][1], ((0,max_x-files[n][1].shape[0]),(0,0)), mode='constant')
            batch_y[n, :] = np.pad(texts[k+n],((0,max_y - texts[k+n].shape[0])), mode='constant')
        _,cost = sess.run([model.optimizer, model.mean_loss],
                          feed_dict={model.X:batch_x, model.Y:batch_y})
    print('epoch %d, loss %f'%(i+1,cost))

Instructions for updating:
Use the `axis` argument instead


minibatch loop:   0%|          | 0/409 [00:00<?, ?it/s]